In [1]:
#https://qiita.com/kenchin110100/items/b34f5106d5a211f4c004

In [2]:
import json
import glob
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
import MeCab
import collections
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import optimizers
from chainer import optimizer
from chainer import serializers

/Users/admin/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def convert(sentence, dictionary):
    return [dictionary[word] if word in dictionary.keys() else -1 for word in sentence]
def process(ids):
    return [end_id] + ids + [null_id] * (th_seq_length - len(ids) - 1)

In [4]:
with open('./temp_data/word2id_dict.pkl','rb') as f:
    word2id_dict = pickle.load(f)
with open('./temp_data/id2word_dict.pkl','rb') as f:
    id2word_dict = pickle.load(f)
with open('./temp_data/questions.pkl','rb') as f:
    questions = pickle.load(f)
with open('./temp_data/answers.pkl','rb') as f:
    answers = pickle.load(f)

In [5]:
m = MeCab.Tagger ("-Owakati")

In [6]:
null_id = 0
end_id = 1
th_seq_length = 20

In [7]:
n_words = len(word2id_dict)

In [8]:
class RNN(chainer.Chain):
    def __init__(self, n_words, n_hiddens):
        super().__init__()
        with self.init_scope():
            self.embed = L.EmbedID(n_words, n_hiddens)
            self.lstm = L.LSTM(n_hiddens, n_hiddens)
            self.fc = L.Linear(n_hiddens, n_words)
            
    def reset_state(self):
        self.lstm.reset_state()

    def get_state(self):
        return self.lstm.c, self.lstm.h
    
    def set_state(self, c, h):
        self.lstm.c = c
        self.lstm.h = h
        
    def __call__(self,x):
        h = self.embed(x)
        h = self.lstm(h)
        h = self.fc(h)
        return h
    

def batch_sampling(x,y,bs):
    r = np.random.permutation(n_seq)[:bs]
    x_batch = x[r]
    y_batch = y[r]
    return x_batch, y_batch

In [9]:
enc = RNN(n_words, 300)
dec = RNN(n_words, 300)

In [10]:
serializers.load_npz('./temp_data/enc_trained',enc)
serializers.load_npz('./temp_data/dec_trained',dec)

In [11]:
bs = 32
ite_pre = 100
ite_seq2seq = 30000

In [19]:
def chat(inp, argmax_decoding = False, print_info = True):
    """
    inp : japanese sentence input
    argmax_decoding : if this is True, this bot reply deterministically
    print_info : if this is True, print input and some information
    """
    inp_wakati = m.parse(inp).split()
    inp_id = convert(inp_wakati, word2id_dict)
    #print(inp_id)
    q = np.array(process(inp_id))

    if print_info:
        if max(np.mean(q == questions,axis=1)) == 1:
            print('training data include this question')
        if -1 in inp_id:
            print('input includes unknown word')
    q = q[::-1]
    
    id_preds = []

    enc.reset_state()
    dec.reset_state()
    for t in range(th_seq_length - 1):
        p = enc(q[t:t+1])

    c,h = enc.get_state()
    dec.set_state(c,h)
    ps = []
    for t in range(1, th_seq_length):
        if t == 1:
            p = dec(np.array([end_id]))
        else:
            p = dec(np.array([pred_id]))
            
        p = F.softmax(p).data[0]
        
        if argmax_decoding == True:
            pred_id = np.argmax(p)
        else:
            pred_id = np.random.choice(range(n_words), p = p)
        p = p[pred_id]
        ps.append(p)
        id_preds.append(int(pred_id))
        if pred_id == null_id:
            break
#     p_joint = 1
#     for p in ps:
#         p_joint *= p
#     print(p_joint ** (1/len(ps)))
    out = ''.join(convert(id_preds[:-1],id2word_dict))
    if print_info:
        print('input  : ', inp)
    print('output : ', out)
    
    return out

In [20]:
inp = 'こんにちは'
out = chat(inp)

training data include this question
input  :  こんにちは
output :  こんにちはー


In [21]:
inp = 'さようなら'
out = chat(inp)

training data include this question
input  :  さようなら
output :  どう日


In [22]:
inp = 'ばいばい'
out = chat(inp)

training data include this question
input  :  ばいばい
output :  ばいばいありい


In [23]:
inp = '東京に遊びに行きませんか？'
out = chat(inp)

input  :  東京に遊びに行きませんか？
output :  加工は見ますか


In [24]:
inp = '世界が平和でありますように'
out = chat(inp)

input  :  世界が平和でありますように
output :  クーラーは心地よいですねですね


In [25]:
inp = '今日は何をして遊びますか？'
out = chat(inp)

input  :  今日は何をして遊びますか？
output :  退屈を紛らわせですか


In [26]:
inp = '何の話ですか？'
out = chat(inp)

training data include this question
input  :  何の話ですか？
output :  需要は全くないです


In [27]:
inp = '需要がないのは良くない'
out = chat(inp)

input  :  需要がないのは良くない
output :  需要？自分にあげ


In [28]:
inp = '熱中症になれよ'
out = chat(inp)

input  :  熱中症になれよ
output :  あなたは対策でれ


In [29]:
inp = 'ブラックな素晴らしきお仕事'
out = chat(inp)

input  :  ブラックな素晴らしきお仕事
output :  パンは買うのです


In [30]:
inp = 'おはようございます'
out = chat(inp)
for k in range(100):
    out = chat(out, print_info=True)

training data include this question
input  :  おはようございます
output :  おは
input  :  おは
output :  もうのよーたー
input  :  もうのよーたー
output :  今日じゃたんですか
input  :  今日じゃたんですか
output :  お昼に何たするのですか
input  :  お昼に何たするのですか
output :  ？から逃げですよねえ
input  :  ？から逃げですよねえ
output :  私は！ではたいにはいですか
input  :  私は！ではたいにはいですか
output :  会社は美味しいですねです
input  :  会社は美味しいですねです
output :  まあうまいが大事
input  :  まあうまいが大事
output :  まあおいしいをおいしいですよ
input  :  まあおいしいをおいしいですよ
output :  通気性が好きですね
input  :  通気性が好きですね
output :  いっぱいいもみたいです
input includes unknown word
input  :  いっぱいいもみたいです
output :  ものを食べるますか
input  :  ものを食べるますか
output :  日傘を言っをたますよ
input  :  日傘を言っをたますよ
output :  それ！ね
input  :  それ！ね
output :  あ
input  :  あ
output :  どうし
input  :  どうし
output :  僕はなったい乱暴ましか
input  :  僕はなったい乱暴ましか
output :  こんにちはに飲みます
input  :  こんにちはに飲みます
output :  今年の
input  :  今年の
output :  サイズが有名のです
input  :  サイズが有名のです
output :  えとかがいい？
input includes unknown word
input  :  えとかがいい？
output :  生が大切ですね
input  :  生が大切ですね
output :  どんな料理、大好きですな好き
input  :  どんな料理、大好き